In [2]:
## Importing required libraries
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
from time import time
from sklearn.preprocessing import normalize

import warnings

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "inputs"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

database.sqlite



In [3]:
start = time()
## Fetching data
#Connecting to database
path = "inputs/"  #Insert path here
database = path + 'database.sqlite'
conn = sqlite3.connect(database)

#Defining the number of jobs to be run in parallel during grid search
n_jobs = 1 #Insert number of parallel jobs here

#Fetching required data tables
player_data = pd.read_sql("SELECT * FROM Player;", conn)
player_stats_data = pd.read_sql("SELECT * FROM Player_Attributes;", conn)
team_data = pd.read_sql("SELECT * FROM Team;", conn)
match_data = pd.read_sql("SELECT * FROM Match WHERE league_id = 21518;", conn)

match_data.head(10)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,21518,21518,21518,2008/2009,1,2008-08-30 00:00:00,530023,10267,8661,3,...,5.00,1.65,3.40,4.50,1.75,3.30,5.00,1.67,3.40,5.00
1,21519,21518,21518,2008/2009,1,2008-08-31 00:00:00,530084,8371,10205,1,...,2.60,2.90,3.20,2.25,2.75,3.20,2.50,2.80,3.25,2.30
2,21520,21518,21518,2008/2009,1,2008-08-31 00:00:00,530085,9783,8633,2,...,1.90,3.25,3.25,2.00,3.50,3.20,2.10,3.25,3.25,2.10
3,21521,21518,21518,2008/2009,1,2008-08-31 00:00:00,530086,8388,8634,1,...,1.57,7.00,3.75,1.45,6.50,3.75,1.53,5.50,3.75,1.53
4,21522,21518,21518,2008/2009,1,2008-08-31 00:00:00,530087,8696,8302,1,...,2.30,2.70,3.10,2.40,2.75,3.25,2.45,2.70,3.25,2.38
5,21523,21518,21518,2008/2009,1,2008-08-31 00:00:00,530088,9869,8305,1,...,2.70,2.50,3.10,2.65,2.50,3.20,2.75,2.60,3.20,2.40
6,21524,21518,21518,2008/2009,1,2008-08-31 00:00:00,530089,8603,8479,0,...,4.50,1.75,3.25,4.35,1.95,3.20,4.00,1.73,3.30,4.50
7,21525,21518,21518,2008/2009,1,2008-08-30 00:00:00,530090,8558,10281,1,...,4.00,1.90,3.25,3.50,2.00,3.20,3.75,1.80,3.25,4.33
8,21526,21518,21518,2008/2009,1,2008-08-31 00:00:00,530091,8315,9865,1,...,4.00,1.90,3.20,3.60,2.00,3.20,3.75,1.91,3.25,3.75
9,21527,21518,21518,2008/2009,1,2008-08-31 00:00:00,530092,9906,9864,4,...,9.00,1.45,3.75,7.00,1.50,3.60,7.50,1.50,3.75,6.00


In [4]:
# Only interesting columns, exclude betting odds for now
matches = match_data[match_data.columns[:11]]
teams = team_data

# Add team names & tidy up
matches = pd.merge(left=matches, right=teams, how='left', left_on='home_team_api_id', right_on='team_api_id')
matches = matches.drop(['country_id','league_id', 'home_team_api_id', 'id_y', 'team_api_id', 'team_short_name'], axis=1)
matches.rename(columns={'id_x':'match_id', 'team_long_name':'home_team'}, inplace=True)
matches = pd.merge(left=matches, right=teams, how='left', left_on='away_team_api_id', right_on='team_api_id')
matches = matches.drop(['id', 'match_api_id', 'away_team_api_id','team_api_id', 'team_short_name'], axis=1)
matches.rename(columns={'id_x':'match_id', 'team_long_name':'away_team'}, inplace=True)

matches.tail()



,match_id,season,stage,date,home_team_goal,away_team_goal,team_fifa_api_id_x,home_team,team_fifa_api_id_y,away_team
3035,24553,2015/2016,9,2015-10-25 00:00:00,2,1,240.0,Atlético Madrid,461.0,Valencia CF
3036,24554,2015/2016,9,2015-10-24 00:00:00,2,0,573.0,Málaga CF,242.0,RC Deportivo de La Coruña
3037,24555,2015/2016,9,2015-10-26 00:00:00,3,0,448.0,Athletic Club de Bilbao,459.0,Real Sporting de Gijón
3038,24556,2015/2016,9,2015-10-24 00:00:00,1,1,110832.0,Granada CF,449.0,Real Betis Balompié
3039,24557,2015/2016,9,2015-10-23 00:00:00,3,0,480.0,Rayo Vallecano,452.0,RCD Espanyol


In [5]:
# TODO: Add data for new seasons (16/17), (17/18) matches
latest_match_data = {}

latest_matches = pd.DataFrame(latest_match_data, columns=['match_id', 'season', 'stage', 'date',
                                                          'home_team_goal', 'away_team_goal',
                                                          'home_team','away_team'])
latest_matches.head(20)

# Add to full training data to predict current season
matches = pd.concat([matches, latest_matches])
matches = matches.reset_index(drop=True)

# Create a full set of match data that can be used with feature engineering later
full_matches = matches.copy()
full_matches.tail(10)

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y
3030,SD Eibar,1,2015-10-25 00:00:00,FC Barcelona,3,24548,2015/2016,9,241.0,467.0
3031,Getafe CF,0,2015-10-24 00:00:00,Sevilla FC,5,24549,2015/2016,9,481.0,1860.0
3032,Villarreal CF,0,2015-10-25 00:00:00,UD Las Palmas,0,24550,2015/2016,9,472.0,483.0
3033,Real Madrid CF,3,2015-10-24 00:00:00,RC Celta de Vigo,1,24551,2015/2016,9,450.0,243.0
3034,Real Sociedad,4,2015-10-25 00:00:00,Levante UD,0,24552,2015/2016,9,1853.0,457.0
3035,Valencia CF,1,2015-10-25 00:00:00,Atlético Madrid,2,24553,2015/2016,9,240.0,461.0
3036,RC Deportivo de La Coruña,0,2015-10-24 00:00:00,Málaga CF,2,24554,2015/2016,9,573.0,242.0
3037,Real Sporting de Gijón,0,2015-10-26 00:00:00,Athletic Club de Bilbao,3,24555,2015/2016,9,448.0,459.0
3038,Real Betis Balompié,1,2015-10-24 00:00:00,Granada CF,1,24556,2015/2016,9,110832.0,449.0
3039,RCD Espanyol,0,2015-10-23 00:00:00,Rayo Vallecano,3,24557,2015/2016,9,480.0,452.0


In [6]:

full_matches['stage'] = full_matches['stage'].apply(pd.to_numeric, errors='coerce')

full_matches.sort_values(by='stage', inplace=True)
full_matches.head(15)

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y
0,RCD Mallorca,0,2008-08-30 00:00:00,Valencia CF,3,21518,2008/2009,1,461.0,453.0
1527,RCD Espanyol,1,2012-08-18 00:00:00,RCD Mallorca,2,23045,2012/2013,1,453.0,452.0
1526,Málaga CF,1,2012-08-18 00:00:00,RC Celta de Vigo,0,23044,2012/2013,1,450.0,573.0
1525,Real Valladolid,1,2012-08-20 00:00:00,Real Zaragoza,0,23043,2012/2013,1,244.0,462.0
1524,Atlético Madrid,1,2012-08-19 00:00:00,Levante UD,1,23042,2012/2013,1,1853.0,240.0
1523,Getafe CF,1,2012-08-18 00:00:00,Sevilla FC,2,23041,2012/2013,1,481.0,1860.0
1522,Real Sociedad,1,2012-08-19 00:00:00,FC Barcelona,5,23040,2012/2013,1,241.0,457.0
1521,Valencia CF,1,2012-08-19 00:00:00,Real Madrid CF,1,23039,2012/2013,1,243.0,461.0
1520,Real Betis Balompié,5,2012-08-19 00:00:00,Athletic Club de Bilbao,3,23038,2012/2013,1,448.0,449.0
1149,RCD Mallorca,1,2012-01-22 00:00:00,Rayo Vallecano,0,22667,2011/2012,1,480.0,453.0


In [7]:
# NEW Functions
from collections import defaultdict
import operator


def game_result(match):
    if match['home_team_goal'] > match['away_team_goal']:
        return 'win'
    elif match['home_team_goal'] < match['away_team_goal']:
        return 'lose'
    else:
        return 'draw'

# - wins/draws/losses til date in season
# - strike of wins/draws/losses til date in season
# - position in league
# - goals favor/against til date in season
# - avg goals per game favor/against til date in season
# - ...
def initialize_season_stats(season_stats,teams):
    for team in teams:
        season_stats[0][team] = {'w_til_date': 0, 'd_til_date': 0, 'l_til_date': 0,
                                  'strike_w_til_date': 0,'strike_d_til_date': 0,'strike_l_til_date': 0,
                                  'points': 0,
                                  'pos': 0,'goals_favor_til_date': 0,'goals_against_til_date': 0,
                                  'avg_goals_favor_til_date': 0, 'avg_goals_against_til_date': 0}

def update_season_stats(season_stats,game):
    prev_stage = game.stage-1
    home_team = game['home_team']
    away_team = game['away_team']
    season_stats[game.stage][home_team] = season_stats[prev_stage][home_team].copy()
    season_stats[game.stage][away_team] = season_stats[prev_stage][away_team].copy()
    # Add results, strikes and points
    result = game_result(game)
    if result == "win":
        #Points
        season_stats[game.stage][home_team]['points'] += 3
        #Results
        season_stats[game.stage][home_team]['w_til_date'] += 1
        season_stats[game.stage][away_team]['l_til_date'] += 1
        #Strikes
        season_stats[game.stage][home_team]['strike_w_til_date'] += 1
        season_stats[game.stage][home_team]['strike_d_til_date'] = 0
        season_stats[game.stage][home_team]['strike_l_til_date'] = 0
        season_stats[game.stage][away_team]['strike_w_til_date'] = 0
        season_stats[game.stage][away_team]['strike_d_til_date'] = 0
        season_stats[game.stage][away_team]['strike_l_til_date'] += 1
    elif result == "draw":
        #Points
        season_stats[game.stage][home_team]['points'] += 1
        season_stats[game.stage][away_team]['points'] += 1
        #Results
        season_stats[game.stage][home_team]['d_til_date'] += 1
        season_stats[game.stage][away_team]['d_til_date'] += 1
        #Strikes
        season_stats[game.stage][home_team]['strike_w_til_date'] = 0
        season_stats[game.stage][home_team]['strike_d_til_date'] +=1
        season_stats[game.stage][home_team]['strike_l_til_date'] = 0
        season_stats[game.stage][away_team]['strike_w_til_date'] = 0
        season_stats[game.stage][away_team]['strike_d_til_date'] += 1
        season_stats[game.stage][away_team]['strike_l_til_date'] = 0
    elif result == "lose":
        #Points
        season_stats[game.stage][away_team]['points'] += 3
        #Results
        season_stats[game.stage][home_team]['l_til_date'] += 1
        season_stats[game.stage][away_team]['w_til_date'] += 1
        #Strikes
        season_stats[game.stage][home_team]['strike_w_til_date'] = 0
        season_stats[game.stage][home_team]['strike_d_til_date'] = 0
        season_stats[game.stage][home_team]['strike_l_til_date'] += 1
        season_stats[game.stage][away_team]['strike_w_til_date'] += 1
        season_stats[game.stage][away_team]['strike_d_til_date'] = 0
        season_stats[game.stage][away_team]['strike_l_til_date'] = 0
    # Add goals and avg
    season_stats[game.stage][home_team]['goals_favor_til_date'] = season_stats[prev_stage][home_team]['goals_favor_til_date'] + game['home_team_goal']
    season_stats[game.stage][away_team]['goals_favor_til_date'] = season_stats[prev_stage][away_team]['goals_favor_til_date'] + game['away_team_goal']
    season_stats[game.stage][home_team]['goals_against_til_date'] = season_stats[prev_stage][home_team]['goals_against_til_date'] + game['away_team_goal']
    season_stats[game.stage][away_team]['goals_against_til_date'] = season_stats[prev_stage][away_team]['goals_against_til_date'] + game['home_team_goal']
    
    season_stats[game.stage][home_team]['avg_goals_favor_til_date'] = season_stats[game.stage][home_team]['goals_favor_til_date'] / game.stage
    season_stats[game.stage][away_team]['avg_goals_favor_til_date'] = season_stats[game.stage][away_team]['goals_favor_til_date'] / game.stage
    season_stats[game.stage][home_team]['avg_goals_against_til_date'] = season_stats[game.stage][home_team]['goals_against_til_date'] / game.stage
    season_stats[game.stage][away_team]['avg_goals_against_til_date'] = season_stats[game.stage][away_team]['goals_against_til_date'] / game.stage
    
        
def set_position_in_league(season_stats,stage):
    prev_stage = stage-1
    sorted_teams = sorted(season_stats[prev_stage].items(), key=lambda y: y[1]['points'])
    for pos,team in enumerate(sorted_teams):
        season_stats[prev_stage][team[0]]['pos'] = pos

# Create a dictionary containing snapshots of the status in each stage
def create_season_stage_stats_til_date(games):
    print("Generating features...")
    stats = defaultdict(dict)
    for season in games.season.unique():
        games_in_season = games[games['season'] == season]
        teams_in_season = games_in_season.home_team.unique()
        season_stats = defaultdict(lambda: defaultdict(dict))
        for stage in games_in_season.stage.unique():
            if stage == 1:
                initialize_season_stats(season_stats,teams_in_season)
            else:
                games_in_previous_stage = games_in_season[games_in_season['stage'] == (stage-1)]
                for i,game in games_in_previous_stage.iterrows():
                    update_season_stats(season_stats,game)
                # Add pos
                set_position_in_league(season_stats,stage)
        stats[season] = season_stats
    print("Done generating features...")
    return stats
    
season_stage_stats_per_team = create_season_stage_stats_til_date(full_matches)



Generating features...
Done generating features...


In [8]:
# Convert home & team into a binary feature, ie Arsenal_h or Arsenal_a
# Need all seasons data for team binary feature
full_match_team_features = pd.DataFrame(full_matches[['season', 'stage','home_team','away_team','home_team_goal','away_team_goal']],
                                   columns=[['season', 'stage','home_team','away_team','home_team_goal','away_team_goal']])

full_match_team_features = pd.concat([full_match_team_features, pd.get_dummies(full_matches['home_team']).rename(columns=lambda x: str(x) + '_h')],
                                axis=1)
full_match_team_features = pd.concat([full_match_team_features, pd.get_dummies(full_matches['away_team']).rename(columns=lambda x: str(x) + '_a')],
                         axis=1)




# Function to determine whether result is a win/draw/lose
def determine_home_result(match):
    if match['home_team_goal'] > match['away_team_goal']:
        return 'win'
    elif match['home_team_goal'] < match['away_team_goal']:
        return 'lose'
    else:
        return 'draw'

#  Add the home team result column to the matches dataframe
full_match_team_features['home_team_result'] = full_match_team_features.apply(determine_home_result, axis=1)
full_match_team_features.tail()

,season,stage,home_team,away_team,home_team_goal,away_team_goal,Athletic Club de Bilbao_h,Atlético Madrid_h,CA Osasuna_h,CD Numancia_h,...,Real Valladolid_a,Real Zaragoza_a,SD Eibar_a,Sevilla FC_a,UD Almería_a,UD Las Palmas_a,Valencia CF_a,Villarreal CF_a,Xerez Club Deportivo_a,home_team_result
311,2008/2009,38,Atlético Madrid,UD Almería,3,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,win
310,2008/2009,38,Valencia CF,Athletic Club de Bilbao,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,win
2210,2013/2014,38,Málaga CF,Levante UD,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,win
315,2008/2009,38,CD Numancia,Sevilla FC,0,2,0,0,0,1,...,0,0,0,1,0,0,0,0,0,lose
698,2009/2010,38,Athletic Club de Bilbao,RC Deportivo de La Coruña,2,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,win


In [9]:
# Add match features based on the computed season/stage/team state dictionary 'season_stage_stats_per_team'

def add_features_for_team(row,away_or_home_str):
    team_stats=season_stage_stats_per_team[row.season][row.stage-1][row[away_or_home_str]]
    row[away_or_home_str+'_w_til_date']=team_stats['w_til_date']
    row[away_or_home_str+'_d_til_date']=team_stats['d_til_date']
    row[away_or_home_str+'_l_til_date']=team_stats['l_til_date']
    row[away_or_home_str+'_strike_w_til_date']=team_stats['strike_w_til_date']
    row[away_or_home_str+'_strike_d_til_date']=team_stats['strike_d_til_date']
    row[away_or_home_str+'_points']=team_stats['points']
    row[away_or_home_str+'_pos']=team_stats['pos']
    row[away_or_home_str+'_goals_favor_til_date']=team_stats['goals_favor_til_date']
    row[away_or_home_str+'_goals_against_til_date']=team_stats['goals_against_til_date']
    row[away_or_home_str+'_avg_goals_favor_til_date']=team_stats['avg_goals_favor_til_date']
    row[away_or_home_str+'_avg_goals_against_til_date']=team_stats['avg_goals_against_til_date']
    
    

def add_match_features(row):
    for t in ['home_team','away_team']:
        add_features_for_team(row,t)
    row['pos_diff']=row['home_team_pos']-row['away_team_pos']
    row['points_diff']=row['home_team_points']-row['away_team_points']
    return row
    

full_match_features_final = full_match_team_features.apply(add_match_features, axis=1)
full_match_features_final.tail(30)

,season,stage,home_team,away_team,home_team_goal,away_team_goal,Athletic Club de Bilbao_h,Atlético Madrid_h,CA Osasuna_h,CD Numancia_h,...,away_team_strike_w_til_date,away_team_strike_d_til_date,away_team_points,away_team_pos,away_team_goals_favor_til_date,away_team_goals_against_til_date,away_team_avg_goals_favor_til_date,away_team_avg_goals_against_til_date,pos_diff,points_diff
2594,2014/2015,38,Rayo Vallecano,Real Sociedad,2,4,0,0,0,0,...,0,0,43,8,40,49,1.081081,1.324324,3,6
2593,2014/2015,38,Granada CF,Atlético Madrid,0,0,0,0,0,0,...,0,0,77,17,67,29,1.810811,0.783784,-14,-43
2592,2014/2015,38,FC Barcelona,RC Deportivo de La Coruña,2,2,0,0,0,0,...,1,0,34,4,33,58,0.891892,1.567568,15,59
2591,2014/2015,38,Levante UD,Elche CF,0,0,0,0,0,0,...,0,0,40,7,35,62,0.945946,1.675676,-2,-4
2590,2014/2015,38,Athletic Club de Bilbao,Villarreal CF,4,0,1,0,0,0,...,2,0,60,14,48,33,1.297297,0.891892,-1,-8
2219,2013/2014,38,Valencia CF,RC Celta de Vigo,2,1,0,0,0,0,...,4,0,49,12,48,52,1.297297,1.405405,-2,-3
2218,2013/2014,38,Real Madrid CF,RCD Espanyol,3,1,0,0,0,0,...,0,1,42,7,40,48,1.081081,1.297297,10,42
2217,2013/2014,38,CA Osasuna,Real Betis Balompié,2,1,0,0,1,0,...,1,0,25,0,35,76,0.945946,2.054054,1,11
2216,2013/2014,38,Real Valladolid,Granada CF,0,1,0,0,0,0,...,0,0,38,3,31,56,0.837838,1.513514,-1,-2
2215,2013/2014,38,UD Almería,Athletic Club de Bilbao,0,0,0,0,0,0,...,0,1,69,16,66,39,1.783784,1.054054,-12,-30


In [14]:
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB



features = full_match_features_final.drop(['season','home_team','away_team','pos_diff','points_diff','home_team_result','home_team_goal','away_team_goal'], axis=1)
target = full_match_features_final['home_team_result']

#TODO: Cross validation
# Make a train/test split using 30% test size
X_train, X_test, y_train, y_test = train_test_split(features, target,
                                                    test_size=0.30,
                                                    random_state=41)

# Fit to data and predict using pipelined GNB and PCA.
unscaled_clf = make_pipeline(MultinomialNB())
unscaled_clf.fit(X_train, y_train)
pred_test = unscaled_clf.predict(X_test)

# Fit to data and predict using pipelined scaling, GNB and PCA.
std_clf = make_pipeline(MinMaxScaler(), MultinomialNB())
std_clf.fit(X_train, y_train)
pred_test_std = std_clf.predict(X_test)

# Show prediction accuracies in scaled and unscaled data.
print('\nPrediction accuracy for the normal test dataset')
print('{:.2%}\n'.format(metrics.accuracy_score(y_test, pred_test)))

print('\nPrediction accuracy for the standardized test dataset')
print('{:.2%}\n'.format(metrics.accuracy_score(y_test, pred_test_std)))

pred_proba_test_std = std_clf.predict_proba(X_test)
print(pred_proba_test_std)
print(pred_test_std)


Prediction accuracy for the normal test dataset
48.14%


Prediction accuracy for the standardized test dataset
52.96%

[[ 0.41329918  0.22014702  0.36655379]
 [ 0.103106    0.14649817  0.75039583]
 [ 0.11645697  0.14487871  0.73866432]
 ..., 
 [ 0.29510932  0.42099912  0.28389157]
 [ 0.33695509  0.30850203  0.35454288]
 [ 0.22319416  0.40757697  0.36922887]]
['draw' 'win' 'win' 'win' 'win' 'lose' 'win' 'win' 'win' 'win' 'win' 'win'
 'win' 'win' 'win' 'win' 'lose' 'win' 'win' 'win' 'lose' 'win' 'win' 'win'
 'win' 'lose' 'win' 'win' 'win' 'win' 'win' 'win' 'win' 'win' 'lose' 'win'
 'win' 'lose' 'win' 'win' 'lose' 'win' 'win' 'win' 'win' 'win' 'draw' 'win'
 'win' 'win' 'lose' 'draw' 'lose' 'win' 'win' 'win' 'lose' 'lose' 'win'
 'win' 'lose' 'win' 'win' 'win' 'win' 'win' 'win' 'win' 'win' 'win' 'lose'
 'win' 'win' 'win' 'win' 'lose' 'lose' 'win' 'win' 'lose' 'win' 'win' 'win'
 'lose' 'win' 'lose' 'lose' 'lose' 'win' 'lose' 'win' 'win' 'win' 'win'
 'win' 'win' 'lose' 'win' 'win' 'win' 'los

In [ ]:
from sklearn.model_selection import GridSearchCV

# GridSearch for finding ideal params


param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']}
 ]
grid_search = GridSearchCV(SVC(), param_grid=param_grid)
grid_search.fit(X_train, y_train)

print(grid_search.best_score_)

In [ ]:
print(grid_search.best_score_)
print(grid_search.best_params_)
print(grid_search.best_estimator_)

In [20]:
# SVC
best_clf = make_pipeline(MinMaxScaler(), SVC(C=10,kernel='linear',probability=True))
best_clf.fit(X_train, y_train)
pred_test_best = best_clf.predict(X_test)
pred_proba_test_best = best_clf.predict_proba(X_test)

print('\nPrediction accuracy for the standardized test dataset with SVC')
print('{:.2%}\n'.format(metrics.accuracy_score(y_test, pred_test_best)))
print(pred_proba_test_best)


Prediction accuracy for the standardized test dataset with SVC
53.40%

[[ 0.31361307  0.23629723  0.4500897 ]
 [ 0.21492559  0.24421333  0.54086108]
 [ 0.23094949  0.23005268  0.53899783]
 ..., 
 [ 0.22541329  0.23527316  0.53931354]
 [ 0.2321384   0.22893761  0.538924  ]
 [ 0.20843546  0.26183071  0.52973383]]
